In [1]:
# basic python data handling analysis modules
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
sns.set()
import pickle, os, gc, re
# small utility functions
from utility import *

# interactive jupyter widgets!
# https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

%matplotlib inline

In [163]:
# import warnings
# warnings.filterwarnings('ignore')
# # cleanliness!

In [162]:
Dataset_directory = ".."+os.sep+"Datasets"+os.sep
tmdb_dir = create_subdir(Dataset_directory, "themoviedb")

In [164]:
# import datetime
# datetime.datetime.now()

In [ ]:
# https://developers.themoviedb.org/3/getting-started/daily-file-exports

# The export job runs every day starting at around 7:00 AM UTC, and all files are available by 8:00 AM UTC.

# http://files.tmdb.org/p/exports/movie_ids_04_28_2017.json.gz

In [28]:
import datetime

utc_now = datetime.datetime.utcnow()

if utc_now.hour<8: # upload for 'today' not necessary donw yet
    utc_now = today - datetime.timedelta(days = 1)
    
movie_ids_file = "http://files.tmdb.org/p/exports/movie_ids_"+utc_now.strftime("%m_%d_%Y")+".json.gz"
movie_ids_file

'http://files.tmdb.org/p/exports/movie_ids_06_26_2019.json.gz'

In [76]:
import shutil
import tempfile
import urllib.request
import gzip

with urllib.request.urlopen(movie_ids_file) as response:
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        shutil.copyfileobj(response, tmp_file)

# with open(tmp_file.name) as html:
#     decompressedFile = gzip.GzipFile(fileobj=html, mode='rb')
#     outFilePath = tmdb_dir + movie_ids_file.split("/")[-1].replace(".gz","")
#     with open(outFilePath, 'w', encoding="utf8") as outfile:
#         outfile.write(decompressedFile.read( 'rt' ))

#     pass


with gzip.open(tmp_file.name, mode='rt', encoding="utf-8") as f_in:
    with open(outFilePath, 'w', encoding="utf-8") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [82]:
movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,original_title,popularity,video
count,467223,467223.000000,467223,467223.000000,467223
unique,2,NaN,429550,NaN,2
top,False,NaN,Carmen,NaN,False
freq,443083,NaN,46,NaN,444777
mean,NaN,331788.498929,NaN,1.111966,NaN
std,NaN,175573.963916,NaN,2.037446,NaN
min,NaN,2.000000,NaN,0.000000,NaN
25%,NaN,191249.000000,NaN,0.600000,NaN
50%,NaN,346870.000000,NaN,0.600000,NaN
75%,NaN,481636.500000,NaN,0.778000,NaN


In [83]:
!pip install tmdbsimple

In [ ]:
# http://api.themoviedb.org/3/movie/16535?api_key=###

In [165]:
# import tmdbsimple as tmdb
text_file = open(tmdb_dir+"my_api_key.txt", "r")
# tmdb.API_KEY = text_file.readlines()[0]
api_key = text_file.readlines()[0]

In [98]:
# with urllib.request.urlopen("http://api.themoviedb.org/3/movie/16535?api_key="+tmdb.API_KEY) as response:
#     resp = response.readlines()
# #     with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
# #         shutil.copyfileobj(response, tmp_file)

In [155]:
import json
import time

tmdb_df = pd.DataFrame(columns = ['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'])

In [177]:
count = 0
fail_ids = []
start_timer = time.time()
for tmdb_id in movie_ids["id"][0:80].values:
    print(tmdb_id)
    try:
        with urllib.request.urlopen("http://api.themoviedb.org/3/movie/"+str(tmdb_id)+"?api_key="+api_key) as response:
            resp = response.readlines()[0]
        # just assuming it doesn't break!

        tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(resp), orient='index').T.values[0]
    except:
        print("failure at id = "+str(tmdb_id))
        fail_ids.append(tmdb_id)
        
    #X-RateLimit
    headers_dict = dict(response.headers.items())
    if 'X-RateLimit-Remaining' in headers_dict.keys():
        rate_limit_remaining = int(headers_dict['X-RateLimit-Remaining'])
        if rate_limit_remaining ==0:
            time.sleep(10)        
        
    count = count +1
    if (count % 40) == 0: # 40 api calls made
        finish_timer = time.time()
        seconds_passed = finish_timer-start_timer
        print("seconds_passed = ",seconds_passed)
        time.sleep( max(10 - seconds_passed,0) ) # respect the rate timer! no more than 40 requests every 10s
        
tmdb_df.to_msgpack( tmdb_dir+"tmdb_df"+".msgpack")

601
602
603
604
605
606
607
608
609
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
seconds_passed =  38.087178230285645
643
644
646
647
648
649
650
651
652
653
654
655
657
658
659
660
failure at id = 660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
685
686
seconds_passed =  74.84928107261658


In [209]:
np.array_split(movie_ids["id"][0:90].values,3)

[array([601, 602, 603, 604, 605, 606, 607, 608, 609, 612, 613, 614, 615,
        616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628,
        629, 630, 631, 632], dtype=int64),
 array([633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 646,
        647, 648, 649, 650, 651, 652, 653, 654, 655, 657, 658, 659, 660,
        661, 662, 663, 664], dtype=int64),
 array([665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677,
        678, 679, 680, 681, 682, 685, 686, 687, 688, 689, 690, 691, 692,
        693, 694, 695, 696], dtype=int64)]

In [215]:
int(np.ceil( len(data)/40 ))

3

In [224]:
batch = tmdb_id_list[ tmdb_id_batch*40:((tmdb_id_batch+1)*40)   ]
print(batch)
URLS = [ "http://api.themoviedb.org/3/movie/"+str(x)+"?api_key="+api_key for x in batch]
URLS

[687 688 689 690 691 692 693 694 695 696]


['http://api.themoviedb.org/3/movie/687?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/688?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/689?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/690?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/691?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/692?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/693?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/694?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/695?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/696?api_key=0c8e2eac3198de39966b29221e7e740b']

In [ ]:
import concurrent.futures
import urllib.request

# URLS = #[some list of urls]
from multiprocessing import Pool
# from multiprocessing.dummy import Pool as ThreadPool


# Retrieve a json string
def load_json_str(url, timeout=60):
    try:
        response = urllib.request.urlopen(url, timeout=timeout)
        output = response.readlines()[0]
    except Exception as exc:
        print('%r generated an exception: %s' % (url, exc))    
#         output = None
        output = None
    return output

# tmdb_id_list = movie_ids["id"][0:90].values
all_movie_ids = movie_ids["id"].values
remaining_movie_ids = list( set(all_movie_ids).difference(set(list(tmdb_df.index))) )

tmdb_id_list  = remaining_movie_ids
# start_timer = time.time()


# Make the Pool of workers
pool = ThreadPool(2)
# Open the urls in their own threads
# and return the results

#close the pool and wait for the work to finish



for tmdb_id_batch in range(0, np.ceil( len(tmdb_id_list)/40 ).astype('int') ):
    batch = tmdb_id_list[ tmdb_id_batch*40:((tmdb_id_batch+1)*40)   ]
#     print(batch)
    
    URLS = [ "http://api.themoviedb.org/3/movie/"+str(x)+"?api_key="+api_key for x in batch]
    
    start_timer = time.time()
    # We can use a with statement to ensure threads are cleaned up promptly
#     with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#         # Start the load operations and mark each future with its URL
#         future_to_url = {executor.submit(load_json_str, url, 60): url for url in URLS}
#         for future in concurrent.futures.as_completed(future_to_url):
#             url = future_to_url[future]
#             try:
#                 data = future.result()
#                 tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(data), orient='index').T.values[0]
#                 # do json processing here
#             except Exception as exc:
#                 print('%r generated an exception: %s' % (url, exc))
# #             else:
# #                 print('%r page is %d bytes' % (url, len(data)))
                
    
    results = pool.map(load_json_str, URLS)
#     for res in results:
#         tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(res), orient='index').T.values[0]
    
#     tmdb_df.loc[batch] =         pd.DataFrame.from_dict( [json.loads(res) for res in results] ).values
    
    df = pd.DataFrame.from_dict( [json.loads(res) for res in results if res is not None] )
    df.index = batch
    tmdb_df.append(df)
    
    finish_timer = time.time()
    duration = finish_timer-start_timer
    print(duration)
    time.sleep( max( 0,12 - duration ) )

    
pool.close()
pool.join()    

5.173295974731445
5.165295362472534
5.522315740585327


In [195]:
#X-RateLimit
headers_dict = dict(response.headers.items())
if 'X-RateLimit-Remaining' in headers_dict.keys():
    rate_limit_remaining = int(headers_dict['X-RateLimit-Remaining'])
    if rate_limit_remaining ==0:
        time.sleep(10)


38

In [197]:
headers_dict = dict(response.headers.items())
headers_dict

{'Access-Control-Allow-Origin': '*',
 'Access-Control-Expose-Headers': 'ETag, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, Retry-After',
 'Cache-Control': 'public, max-age=28800',
 'Content-Type': 'application/json;charset=utf-8',
 'Date': 'Fri, 28 Jun 2019 10:43:35 GMT',
 'ETag': '"0bfb58d5beab35eee92118b7ead367fd"',
 'Server': 'openresty',
 'Vary': 'Accept-Encoding',
 'X-Memc': 'HIT',
 'X-Memc-Age': '16716',
 'X-Memc-Expires': '12084',
 'X-Memc-Key': 'c9ddf14d6f25e0d78cbfda351f680cc9',
 'X-RateLimit-Limit': '40',
 'X-RateLimit-Remaining': '38',
 'X-RateLimit-Reset': '1561718624',
 'Content-Length': '1267',
 'Connection': 'Close'}

In [168]:
tmdb_df.shape

(153637, 25)

In [169]:
tmdb_df.to_msgpack( tmdb_dir+"tmdb_df_why_slow"+".msgpack")

In [153]:
((467223/40)*10) / 3600

32.446041666666666

In [171]:
len(movie_ids["id"][1200:].values)

466023

In [173]:
(466023/3)/3600

43.15027777777778

In [161]:
# 1200 -> ~1MB
467223/1200
# -> 400MB

389.3525

In [ ]:
# 32 hours starting ~ 17ish Wednesday -> 1am Friday morning